<a href="https://colab.research.google.com/github/MINDS-edu/The-5th-POSTECH-Youth-Mathematical-Artificial-Intelligence-Academy/blob/master/PYMAIA5_day3_CNN_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 수업 전체 순서: MLP vs CNN 손글씨 인식 실습

3주차 실습 수업에서는 MLP(다층 퍼셉트론)과 CNN(합성곱 신경망)의 차이를 직접 코드로 실험해보며 이해합니다.  
마지막에는 내가 그린 손글씨 숫자를 두 모델에 입력해 비교합니다.

# Part 1. MNIST 손글씨 데이터 살펴보기

## 🎯 목표
- 우리가 사용할 손글씨 숫자 데이터셋(MNIST)가 어떤 모습인지 직접 눈으로 확인한다.
- 신경망을 돌리기 전에, 데이터를 먼저 이해하는 습관을 만든다.

## 🧾 MNIST란?
- 0부터 9까지의 손글씨 숫자 이미지로 이루어진 데이터셋.
- 각 이미지는 **28×28 픽셀**, 흑백 이미지.
- 학습용 60,000장, 테스트용 10,000장으로 구성된다.

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist

# 1. 데이터 불러오기
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)

# 2. 임의의 100개 데이터 인덱스 선택
indices = np.random.choice(len(x_train), size=100, replace=False)

# 3. 10x10 격자로 이미지 출력
fig, axes = plt.subplots(10, 10, figsize=(10, 10))
for i, index in enumerate(indices):
    row = i // 10
    col = i % 10
    axes[row, col].imshow(x_train[index], cmap='gray')
    axes[row, col].axis('off')
plt.suptitle("MNIST Training Data (100 samples)", fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, Sequential
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
(x_train, y_train), (x_test, y_test) = mnist.load_data()
i = 775# @param {type:"raw"}
plt.imshow(x_train[i], cmap='gray')
plt.title(f"Label: {y_train[i]}")

# Part 2. MLP(다층 퍼셉트론)으로 손글씨 인식해보기

## 🎯 목표
- MLP(다층 퍼셉트론)가 무엇인지 이해한다.
- MNIST 숫자를 MLP에 넣어서 분류하고, 정확도를 확인한다.

## 🧠 MLP(Multi-Layer Perceptron)란?

MLP는 **입력층 → 여러 개의 은닉층 → 출력층**으로 구성된 가장 기본적인 형태의 인공신경망입니다. 모든 노드가 다음 층의 모든 노드와 연결되는 **완전연결(fully connected)** 구조를 사용합니다. 특히 이미지를 입력으로 사용할 때는 2차원 이미지를 그대로 쓰지 않고, 예를 들어 **28×28 이미지 → 길이 784의 벡터**로 **평탄화(flatten)** 해서 MLP에 넣습니다.

## 🔧 MLP의 동작 방식

### 1️⃣ 여러 개의 다른 선형함수를 만들고 각각 활성화 함수 적용
각 노드 $h_i$는 입력 $x$에 대해 서로 다른 선형함수

$$
z_i = a_i x + b_i
$$

를 만들고, 여기에 활성화 함수 $\sigma(z_i)$를 적용합니다.

<img src=https://raw.githubusercontent.com/MINDS-edu/The-5th-POSTECH-Youth-Mathematical-Artificial-Intelligence-Academy/refs/heads/master/warehouse/first_layer.gif height=300px;>

### 2️⃣ 활성화 함수를 통과한 값들을 다시 다른 선형함수에 입력
첫 번째 층의 출력을 벡터로 묶은 뒤, 다음 층에서 다시 선형변환 → 활성화 과정을 반복합니다.
<img src=https://raw.githubusercontent.com/MINDS-edu/The-5th-POSTECH-Youth-Mathematical-Artificial-Intelligence-Academy/refs/heads/master/warehouse/second_third_layer.gif height=300px;>

이 과정을 여러 번 반복하면 깊은 신경망(Deep Neural Network)이 됩니다.

## ⚠️ MLP의 한계점

MLP는 이미지 입력을 **펼쳐서(flatten)** 사용하기 때문에 중요한 문제점이 있습니다.

- **공간적 구조(spatial structure)가 완전히 사라짐**
  - 어떤 픽셀이 왼쪽/오른쪽, 위/아래에 있는지 관계를 알 수 없음
- 따라서 다음과 같은 특징을 스스로 이해하기 어렵다.
  - “획이 연결되어 있다”
  - “대각선 모양이다”
  - “가까운 픽셀끼리 의미가 있다”

→ 이런 이유로 이미지 처리에는 CNN(Convolutional Neural Network)이 등장했습니다.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical

# 1. 데이터 정규화 (0~255 -> 0~1)
x_train_norm = x_train.astype('float32') / 255.0
x_test_norm  = x_test.astype('float32') / 255.0

# 2. (28,28) -> (784,)
x_train_mlp = x_train_norm.reshape(-1, 28*28)
x_test_mlp  = x_test_norm.reshape(-1, 28*28)

# 3. 레이블을 one-hot 벡터로 변환 (예: 3 -> [0,0,0,1,0,0,0,0,0,0])
y_train_cat = to_categorical(y_train, 10)
y_test_cat  = to_categorical(y_test, 10)

# 4. MLP 모델 정의
mlp = Sequential()
mlp.add(Dense(128, activation='relu', input_shape=(784,)))
mlp.add(Dense(64, activation='relu'))
mlp.add(Dense(10, activation='softmax'))

mlp.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 5. 학습
history_mlp = mlp.fit(
    x_train_mlp, y_train_cat,
    epochs=5,
    batch_size=128,
    validation_data=(x_test_mlp, y_test_cat),
    verbose=1
)

# 6. 성능 평가
mlp_loss, mlp_acc = mlp.evaluate(x_test_mlp, y_test_cat, verbose=0)
print("MLP Test Accuracy:", mlp_acc)

# 7. 학습 곡선 확인
plt.plot(history_mlp.history['loss'], label='Train Loss')
plt.plot(history_mlp.history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('MLP Loss Curve')
plt.legend()
plt.show()

# Part 3. CNN(합성곱 신경망)으로 손글씨 인식해보기

## 🎯 목표
- CNN(합성곱 신경망)이 무엇인지 이해한다.
- CNN이 MLP와 **어떻게 다른지** 직관적으로 이해한다.
- MNIST 숫자를 CNN에 넣어 분류하고, MLP와 **성능을 비교**해 본다.

## 🧠 CNN(Convolutional Neural Network)란?

CNN은 이미지처럼 **가로·세로 구조가 있는 데이터**를 잘 처리하기 위해 만들어진 신경망입니다.

- MLP는 28×28 이미지를 **그냥 784차원 벡터**로 펼쳐서 사용합니다.  
  → 픽셀 사이의 **공간적인 이웃 관계(위/아래, 왼쪽/오른쪽)** 정보를 전혀 쓰지 못합니다.
- CNN은 다음 두 가지 아이디어를 이용해서 이미지를 더 똑똑하게 봅니다.
  1. **작은 필터(커널)를 사용하는 합성곱(Convolution)**
  2. **MaxPooling을 이용한 다운샘플링**

즉, CNN은 “이미지 위에서 작은 창(필터)을 움직이며 패턴을 찾는 네트워크”라고 생각하면 됩니다.

![](https://i.sstatic.net/9bi5k.gif)

## 🔧 CNN의 동작 방식

### 1️⃣ Conv2D(합성곱 층): 작은 필터로 주변 패턴 뽑기

이미지 한 장을 생각해 봅시다 (28×28 픽셀).

- CNN은 **3×3 같은 작은 필터**를 이미지 위에서 슬라이딩하면서
  - “세로선”, “가로선”, “대각선”, “모서리”, “곡선” 같은 **로컬 패턴**을 찾아냅니다.
- 하나의 Conv 층에는 이런 필터가 여러 개 있습니다.
  - 예: `Conv2D(32, kernel_size=(3, 3))`
    - 3×3 필터를 32개 가지고 있고,
    - 각각이 서로 다른 패턴(예: 가로줄, 세로줄, 모서리, 작은 곡선 등)을 학습합니다.

### 2️⃣ MaxPooling2D: 크기를 줄이고 위치 변화에 강하게 만들기

합성곱 층을 지난 결과(특징 맵)는 여전히 2차원 구조를 갖는 큰 배열입니다.  
이 상태에서 **MaxPooling**을 적용하면:

- 예를 들어 2×2 영역에서 **가장 큰 값 하나만 선택**합니다.
- 결과:
  - 이미지 크기가 줄어들어 → **계산량 감소**
  - 조금 위치가 달라도 큰 값만 남기기 때문에  
    → **작은 이동(translation)에 덜 민감**해집니다.

즉, “디테일을 조금 버리는 대신, **핵심 특징만 압축해서 들고 간다**”라고 볼 수 있습니다.

### 3️⃣ Conv + Pooling을 여러 번 반복하기

일반적인 CNN 구조에서는:

1. 합성곱 → 풀링  
2. 다시 합성곱 → 풀링

이 과정을 여러 번 반복합니다.

- 앞쪽 층: **간단한 패턴**(선, 모서리)을 보고,
- 뒤쪽 층: 그 패턴들을 조합해서 **복잡한 패턴**(숫자의 일부, 특정 획 조합)을 봅니다.

이 과정이 반복되면,
- “그냥 픽셀 뭉치”였던 이미지가
- “유용한 특징 벡터”로 점점 변환됩니다.

### 4️⃣ Flatten + Dense 층: 최종적으로 숫자(0~9) 분류하기

합성곱과 풀링을 거친 뒤에는 여전히 2차원(또는 3차원) 형태의 특징 맵(feature map)입니다.

- **Flatten**
  - 2D/3D 특징 맵을 **1차원 벡터**로 펼칩니다.
- **Dense(완전연결 층)**
  - 펼쳐진 벡터를 이용해 숫자 분류에 필요한 고차원 특징을 조합합니다.
- **Softmax 출력층**
  - 출력이 길이 10인 벡터가 되고,
  - 각 원소가 “이 이미지가 숫자 0~9일 **확률**”을 의미합니다.

예를 들어 출력이

\[
0.01, 0.02, 0.90, 0.03, 0.01, 0.01, 0.01, 0.00, 0.00, 0.01
\]

이라면, “이 이미지는 숫자 **2**일 확률이 가장 높다”고 해석합니다.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# 1) 데이터 불러오기 (MNIST)
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 2) CNN 입력 형태에 맞게 전처리 (28, 28, 1) + 정규화
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32')  / 255.0

x_train = x_train.reshape(-1, 28, 28, 1)
x_test  = x_test.reshape(-1, 28, 28, 1)

# 3) 레이블을 one-hot 벡터로 변환 (0~9 → 길이 10짜리 벡터)
y_train = to_categorical(y_train, 10)
y_test  = to_categorical(y_test, 10)

# 4) CNN 모델 정의
model = Sequential()
# 첫 번째 합성곱 층: 3x3 필터 32개, ReLU 활성화
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28, 28, 1)))
# 2x2 맥스 풀링
model.add(MaxPooling2D(pool_size=(2, 2)))

# 두 번째 합성곱 층: 3x3 필터 64개, ReLU
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
# 다시 2x2 맥스 풀링
model.add(MaxPooling2D(pool_size=(2, 2)))

# 특징맵을 1차원으로 펼치기
model.add(Flatten())

# 완전연결층 128개 노드 + ReLU
model.add(Dense(128, activation='relu'))

# 출력층: 10개 노드 + softmax (0~9 확률)
model.add(Dense(10, activation='softmax'))

# 5) 모델 컴파일: 다중 클래스 분류 → categorical_crossentropy
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# 6) 모델 학습
history = model.fit(
    x_train, y_train,
    epochs=4,
    batch_size=128,
    validation_data=(x_test, y_test),
    verbose=1
)

# 7) 테스트 데이터로 최종 성능 평가
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

# 8) 학습 과정에서의 loss 변화 그리기
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('CNN Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()


In [ ]:
from IPython.display import HTML, display
import base64
from PIL import Image
from google.colab import output

# HTML + JavaScript 코드
canvas_html = """
<canvas id="canvas" width="280" height="280" style="border:1px solid black;"></canvas><br>
<button onclick="saveCanvas()">Save as PNG</button>
<button onclick="clearCanvas()">Clear Canvas</button>
<script>
let canvas = document.getElementById('canvas');
let ctx = canvas.getContext('2d');
let drawing = false;

canvas.addEventListener('mousedown', (e) => { drawing = true; ctx.beginPath(); });
canvas.addEventListener('mouseup', () => { drawing = false; });
canvas.addEventListener('mousemove', (e) => {
    if (!drawing) return;
    ctx.lineWidth = 10;
    ctx.lineCap = 'round';
    ctx.strokeStyle = 'black';
    let rect = canvas.getBoundingClientRect();
    ctx.lineTo(e.clientX - rect.left, e.clientY - rect.top);
    ctx.stroke();
});

function saveCanvas() {
    let dataURL = canvas.toDataURL('image/png');
    google.colab.kernel.invokeFunction('notebook.save_canvas', [dataURL], {});
}

function clearCanvas() {
    ctx.clearRect(0, 0, canvas.width, canvas.height);
}
</script>
"""

# 데이터를 저장하는 Python 콜백 함수
def save_canvas(data):
    png_data = base64.b64decode(data.split(',')[1])
    with open('canvas.png', 'wb') as f:
        f.write(png_data)
    print("Saved as canvas.png")


# Colab 콜백 함수 등록
output.register_callback('notebook.save_canvas', save_canvas)

# HTML 렌더링
display(HTML(canvas_html))

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

image = Image.open('canvas.png').convert('RGBA')
image = np.array(image.resize((28, 28))).astype('float32') / 255.0
image = image[:, :, 3]  # 알파 채널

fig, axs = plt.subplots(1, 3, figsize=(10, 4))
plt.sca(axs[0])
plt.imshow(image, cmap='gray')
plt.axis('off')
plt.title('Input')

tns = ["CNN", "MLP"]
categories = [str(i) for i in range(10)]

for im in range(2):
    plt.sca(axs[im+1])
    if im == 0:
        X = image.reshape(1, 28, 28, 1)
        pred = model.predict(X)[0]
    else:
        X = image.reshape(1, 28*28)
        pred = mlp.predict(X)[0]

    plt.bar(categories, pred)
    plt.ylim(0, 1.0)
    plt.title(tns[im])
    plt.xlabel('Categories')
    plt.ylabel('Probability')

plt.tight_layout()
plt.show()